##Scrape Wikipedia for all the movies and their information on box office and budget.##
We can clean result for the budget, but the result for the box office is too difficult to clean. We will get box office information from other sources.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
import re
import string
import json
from tqdm import tqdm

Scrape wikipedia page of each Oscar into yearstext dictionary

In [2]:
SUFFIXES = {1: 'st', 2: 'nd', 3: 'rd'}
def ordinal(num):
    if 10 <= num % 100 <= 20:
        suffix = 'th'
    else:
        suffix = SUFFIXES.get(num % 10, 'th')
    return str(num) + suffix

In [3]:
yearstext={}
for year in range(1,88):
    wikipage="https://en.wikipedia.org/wiki/{}_Academy_Awards".format(ordinal(year))
    if requests.get(wikipage).status_code==404:
        print year
    else:
        yearstext[year] = requests.get(wikipage)
    

In [4]:
#yearstext

Now we create a function to get the movie urls from each Oscar wikipedia page.

In [6]:
#test
year=70
yeartextdict = yearstext[year]
soup = BeautifulSoup(yeartextdict.text, "html.parser")
rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[0:]
if year in range(9,59)+[69]:
    s1=1
    s2=3
    s3=5
else:
    s1=0
    s2=1
    s3=2
bp_text=rows[s1].find_all("td")[0]
bd_text=rows[s1].find_all("td")[1]
b_actor_text=rows[s2].find_all("td")[0]
b_actress_text=rows[s2].find_all("td")[1]
b_s_actor_text=rows[s3].find_all("td")[0]
b_s_actress_text=rows[s3].find_all("td")[1]
bp_text.find_all("li")[0].find_all("i")[0].find("a").get_text()


u'Titanic'

This function applies to each Oscar wikipedia page for the 9th to 59th Oscars. It returns the movie urls and the movie titles and a dictionary with keys as movie urls and value as movie titles.

In [7]:
#function to get movie url for the 9th to 59th Oscars
def get_movie_url_2(year, yearstext):
    yeartextdict = yearstext[year]
    soup = BeautifulSoup(yeartextdict.text, "html.parser")
    rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[0:]
    if year in range(9,59)+[69]:
        s1=1
        s2=3
        s3=5
    else:
        s1=0
        s2=1
        s3=2
    bp_text=rows[s1].find_all("td")[0]
    bd_text=rows[s1].find_all("td")[1]
    b_actor_text=rows[s2].find_all("td")[0]
    b_actress_text=rows[s2].find_all("td")[1]
    b_s_actor_text=rows[s3].find_all("td")[0]
    b_s_actress_text=rows[s3].find_all("td")[1]
    movie_url=[]
    movie_title=[]
    for m in bp_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in bd_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_actor_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_actress_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_s_actor_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_s_actress_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    movies=dict(zip(movie_url,movie_title))
    return movie_url , movie_title, movies
    

This function applies to each Oscar wikipedia page for the 9th to 59th Oscars. It returns the movie urls and the movie titles and a dictionary with keys as movie urls and value as movie titles.

In [8]:
#function to get movie url for first 9 Oscars
def get_movie_url_1(year, yearstext):
    yeartextdict = yearstext[year]
    soup = BeautifulSoup(yeartextdict.text, "html.parser")
    rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[0:]
    if year ==1 :
        s1=0
        s2=1
        s3=2
        bp_text=rows[s1].find_all("td")[0]
        bd_text=rows[s2].find_all("td")[0]
        bd_text2=rows[s2].find_all("td")[1]
        b_actor_text=rows[s3].find_all("td")[0]
        b_actress_text=rows[s3].find_all("td")[1]
        #b_s_actor_text=rows[s3].find_all("td")[0]
        #b_s_actress_text=rows[s3].find_all("td")[1]
        movie_url=[]
        movie_title=[]
        for m in bp_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in bd_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in bd_text2.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actor_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actress_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
                
    else :
        s1=1
        s2=3
        bp_text=rows[s1].find_all("td")[0]
        bd_text=rows[s1].find_all("td")[1]
        b_actor_text=rows[s2].find_all("td")[0]
        b_actress_text=rows[s2].find_all("td")[1]
        #b_s_actor_text=rows[s3].find_all("td")[0]
        #b_s_actress_text=rows[s3].find_all("td")[1]
        movie_url=[]
        movie_title=[]
        for m in bp_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in bd_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actor_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actress_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
    movies=dict(zip(movie_url,movie_title))
    return movie_url , movie_title, movies

In [10]:
#get_movie_url_2(80,yearstext)[0]

Apply the function to all the wikipedia Oscar pages.

In [11]:
movie_url=[]
movie_titles=[]
movie_dict={} #movie_url as key -- movie title
for year in range(1,10):
    movie_url.extend(get_movie_url_1(year,yearstext)[0])
    movie_titles.extend(get_movie_url_1(year,yearstext)[1])
    movie_dict.update(get_movie_url_1(year,yearstext)[2])
for year in range(9,88):
    #print year
    movie_url.extend(get_movie_url_2(year,yearstext)[0])
    movie_titles.extend(get_movie_url_2(year,yearstext)[1])
    movie_dict.update(get_movie_url_2(year,yearstext)[2])

Check that the lenths are as desired.

In [12]:
movie_url=list(set(movie_url))
movie_titles=list(set(movie_url))
#movie_dict
len(movie_url),len(movie_titles),len(movie_dict)

(1275, 1275, 1275)

Scrape wikipedia to get all the movie wikipedia pages. These are stored into ```moviestex``` dictionary. In case of failed ```get.``` attempt, we store all the failed urls into the ```failed_url``` list. It turns out that all the ```get.``` attemps are successful.

In [ ]:
moviestext={}
failed_url=[]
for url in movie_url:
    wikipage="https://en.wikipedia.org{}".format(url)
    if requests.get(wikipage).status_code==404:
        failed_url.append(url)
    else:
        moviestext[url] = requests.get(wikipage)
        time.sleep=1.0

Exception in thread Thread-1:
Traceback (most recent call last):
  File "/Users/annie/anaconda/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/Users/annie/anaconda/lib/python2.7/site-packages/IPython/kernel/zmq/parentpoller.py", line 37, in run
    time.sleep(1.0)
TypeError: 'float' object is not callable



In [ ]:
len(moviestext),len(failed_url)

(1275, 0)

In [ ]:
movietextdict=moviestext[moviestext.keys()[0]]

We now write a function for each movie page. This function takes ```movie_url``` and ```moviestext``` (the dictionary containing all scraped text) and returns the budget and box office information in list forms.

In [ ]:
def get_budget(movie_url,moviestext):
    movietextdict=moviestext[movie_url]
    bad_movie=[]
    budget=["NA"]
    box_office=["NA"]
    soup = BeautifulSoup(movietextdict.text, "html.parser")
    if soup.find("table", attrs={"class": "infobox vevent"})==None:
        bad_movie.append(movie_url)
    else:
        rows = soup.find("table", attrs={"class": "infobox vevent"}).find_all("tr")[0:]
        for r in rows:
            if r.find("th")==None:
                continue
            if r.find("th").get_text()=="Budget":
                budget=[]
                budget=[r.find("td").get_text().split("[")[0]]

            if r.find("th")==None:
                continue
            if r.find("th").get_text()=="Box office":
                box_office=[]
                for b in r.find("td").get_text().split("\n"):
                    box_office.append(b.split("[")[0])
    
    return budget,box_office
        


In [ ]:
##test
get_budget(moviestext.keys()[0],moviestext)

([u'A$6 million'], [u'A$2,898,000 (Australia)', u'US$10,278,575'])

Apply the function and check the resulting lengths.

In [ ]:
movieurl_budget_dict={}
movieurl_bo_dict={}
for movieurl in moviestext.keys():
    #print movieurl
    movieurl_budget_dict[movieurl],movieurl_bo_dict[movieurl]=get_budget(movieurl,moviestext)


In [ ]:
#movieurl_budget_dict

In [ ]:
#movieurl_bo_dict
len(movieurl_budget_dict),len(movieurl_bo_dict)

(1275, 1275)

Write a function takes in each movie page and return the list of imdb link ```imdb_link``` and list of rotten tomatoes link ```rt_link``` .

In [ ]:
movietextdict=moviestext[moviestext.keys()[0]]

In [ ]:
###test
'''soup = BeautifulSoup(movietextdict.text, "html.parser")
#rows=soup.find_parents("")[0:][-1].next_element.next_element
rows=soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]
#rows=soup.find_all("table")[0:] #,attrs={"class": "mbox-small plainlinks sistersitebox"}
external_links=[]
for row in rows:
    if row.find("i")!= None:
        external_links.append(row.get("href"))

for l in external_links:
    if "imdb" in l:
        imdb_link=l
    if "rottentomatoes" in l:
        rt_link=l
'''

'soup = BeautifulSoup(movietextdict.text, "html.parser")\n#rows=soup.find_parents("")[0:][-1].next_element.next_element\nrows=soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]\n#rows=soup.find_all("table")[0:] #,attrs={"class": "mbox-small plainlinks sistersitebox"}\nexternal_links=[]\nfor row in rows:\n    if row.find("i")!= None:\n        external_links.append(row.get("href"))\n\nfor l in external_links:\n    if "imdb" in l:\n        imdb_link=l\n    if "rottentomatoes" in l:\n        rt_link=l\n'

In [ ]:
#function to get imbd movie id and rotten tomatoes url
def get_imbd_rt_url(movie_url,moviestext):
    movietextdict=moviestext[movie_url]
    soup = BeautifulSoup(movietextdict.text, "html.parser")
    rows=soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]
    external_links=[]
    imdb_link="NA"
    rt_link="NA"
    for row in rows:
        if row.find("i")!= None:
            external_links.append(row.get("href"))

    for l in external_links:
        if "imdb" in l:
            imdb_link=l
        if "rottentomatoes" in l:
            rt_link=l
    return imdb_link,rt_link
    

In [ ]:
#test
get_imbd_rt_url(moviestext.keys()[0],moviestext)

(u'http://www.imdb.com/title/tt0086617/',
 u'http://www.rottentomatoes.com/m/year_of_living_dangerously/')

Apply the function to all movies.

In [ ]:
movieurl_imdb_link_dict={}
movieurl_rt_link_dict={}
for movieurl in moviestext.keys():
    #print movieurl
    movieurl_imdb_link_dict[movieurl],movieurl_rt_link_dict[movieurl]=get_imbd_rt_url(movieurl,moviestext)

In [ ]:
len(movieurl_imdb_link_dict),len(movieurl_rt_link_dict)

(1275, 1275)

Merge into dataframe.

In [ ]:
movie_url_df=pd.DataFrame(movie_dict.items(),columns=['movie_url', 'title'])
movie_budget_df=pd.DataFrame(movieurl_budget_dict.items(),columns=['movie_url', 'budget'])
movie_bo_df=pd.DataFrame(movieurl_bo_dict.items(),columns=['movie_url', 'box_office'])
movie_imdb_link_df=pd.DataFrame(movieurl_imdb_link_dict.items(),columns=['movie_url', 'imdb_link'])
movie_rt_link_df=pd.DataFrame(movieurl_rt_link_dict.items(),columns=['movie_url', 'rt_link'])
#movie_budget_df.head()
#movie_bo_df.head()
#movie_url_df.head()
#movie_imdb_link_df.head()

In [ ]:
frames = [movie_url_df, movie_budget_df, movie_bo_df]
df = pd.merge(movie_url_df, movie_budget_df, on='movie_url')
df=pd.merge(df, movie_bo_df, on='movie_url')
df=pd.merge(df, movie_imdb_link_df, on='movie_url')
df=pd.merge(df, movie_rt_link_df, on='movie_url')
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/


In [ ]:
print "movies without rt_link"  
print df[df["rt_link"]=="NA"].shape[1]

movies without rt_link
6


In [ ]:
df.to_pickle('wiki_df_raw.csv')